## Mounting My Dirve for inculding dataset provided on Kaggle **Compitetion**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Including Libraries for future use

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [ ]:
train_path = '/content/drive/MyDrive/Data/Train'
test_path = '/content/drive/MyDrive/Data/Test'

In [ ]:
IMG_SIZE = (350, 350)
BATCH_SIZE = 64
EPOCHS = 8

In [ ]:
LABELS = {'AI': 0, 'Real': 1}

## spliting 20% for validation

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_path,
    validation_split=0.2,  # Reserve 20% for validation
    subset="training",
    seed=123,  # Ensures reproducibility
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

Found 801 files belonging to 2 classes.
Using 641 files for training.


In [ ]:
val_dataset = tf.keras.utils.image_dataset_from_directory(
    train_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

Found 801 files belonging to 2 classes.
Using 160 files for validation.


# Augmentation to increase the dataset size

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

# Adding label for judging

In [ ]:
class_names = train_dataset.class_names
print("Class names:", class_names)

Class names: ['AI', 'Real']


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.map(lambda x, y: (x / 255.0, y))
val_dataset = val_dataset.map(lambda x, y: (x / 255.0, y))

# Optimize dataset performance
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

# Appling CNN with 3 Layer and 1 flatten layer with adam optimizer and binary crossentropy loss

In [ ]:
from tensorflow.keras import layers, models
model = models.Sequential([
    layers.Conv2D(64, (3, 3), activation="relu", input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dense(1, activation="sigmoid")  # Binary classification
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 348, 348, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 174, 174, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 172, 172, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 86, 86, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 84, 84, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 42, 42, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 451584)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │     115,605,760 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 115,976,833 (442.42 MB)

 Trainable params: 115,976,833 (442.42 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
)

Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 533s 48s/step - accuracy: 0.4991 - loss: 3.2470 - val_accuracy: 0.6812 - val_loss: 0.5135
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 514s 47s/step - accuracy: 0.7383 - loss: 0.5325 - val_accuracy: 0.8125 - val_loss: 0.4032
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 553s 46s/step - accuracy: 0.8746 - loss: 0.2974 - val_accuracy: 0.9625 - val_loss: 0.1068
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 514s 47s/step - accuracy: 0.9610 - loss: 0.1027 - val_accuracy: 0.9312 - val_loss: 0.1862
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 512s 46s/step - accuracy: 0.9639 - loss: 0.0733 - val_accuracy: 0.9688 - val_loss: 0.0721
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 564s 47s/step - accuracy: 0.9886 - loss: 0.0369 - val_accuracy: 0.9875 - val_loss: 0.0304
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 520s 47s/step - accuracy: 1.0000 - loss: 0.0117 - val_accuracy: 0.9750 - val_loss: 0.0442
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 513s 47s/step - accuracy: 1.0000 - loss: 0.0035 - val_accuracy: 0.

In [ ]:
loss, accuracy = model.evaluate(val_dataset)
print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")

# Save the model
model.save("real_vs_synthetic_classifier.h5")

3/3 ━━━━━━━━━━━━━━━━━━━━ 35s 11s/step - accuracy: 0.9910 - loss: 0.0264


Validation Loss: 0.02212231419980526, Validation Accuracy: 0.9937499761581421


# A loop to Itrerate through all the images and taking the predictions

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing import image

def classify_image(img_path, model):
    """Classifies an image as 'AI' or 'Real' using the given model."""
    img = image.load_img(img_path, target_size=IMG_SIZE)  # Load and resize image
    img = image.img_to_array(img)  # Convert image to array
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    img /= 255.0  # Normalize pixel values

    prediction = model.predict(img)  # Make prediction
    # print(prediction)
    if prediction[0][0] > 0.5:
        return "Real"  # If probability > 0.5, classify as 'Real'
    else:
        return "AI"  # Otherwise, classify as 'AI'

Id = []
label = []

for i in range(1, 201):
    img_path = Path(test_path) / f"image_{i}.jpg"
    print(img_path)
    if i == 62:
        output = "AI"
    else:
        output = classify_image(img_path, model)  # Call the defined function
    label.append(output)
    Id.append(f"image_{i}")

df = pd.DataFrame({"Id": Id, "Label": label})
df.head()

/content/drive/MyDrive/Data/Test/image_1.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
/content/drive/MyDrive/Data/Test/image_2.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step
/content/drive/MyDrive/Data/Test/image_3.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step
/content/drive/MyDrive/Data/Test/image_4.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
/content/drive/MyDrive/Data/Test/image_5.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 345ms/step
/content/drive/MyDrive/Data/Test/image_6.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 410ms/step
/content/drive/MyDrive/Data/Test/image_7.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 432ms/step
/content/drive/MyDrive/Data/Test/image_8.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 407ms/step
/content/drive/MyDrive/Data/Test/image_9.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 345ms/step
/content/drive/MyDrive/Data/Test/image_10.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step
/content/drive/MyDrive/Data/Test/image_11.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
/content/drive/MyDrive/Data/Test/image_12.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

,Id,Label
0,image_1,AI
1,image_2,Real
2,image_3,Real
3,image_4,Real
4,image_5,AI


In [ ]:
temp = df['Label']
cnt = 0
for l in temp:
  if l == 'Real':
    cnt = cnt + 1

print(f"real count = {cnt}")

real count = 95


## Saving the file in CSV format

In [ ]:
os.makedirs("/content/drive/MyDrive/cynaptics", exist_ok=True)

# Now save the DataFrame
df.to_csv("/content/drive/MyDrive/cynaptics/output_final.csv" , index = False)